<a href="https://colab.research.google.com/github/yyic1203/Proyectos/blob/main/AvanzadoI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import numpy as np
from scipy.linalg import eigh


def autovector(FP, n):
    # Calculate eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eigh(FP)
    return eigenvectors

def find_X_and_X_dagger(S, n):
    # Calculate eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eigh(S)
    D = np.diag(1.0 / np.sqrt(eigenvalues))
    X = np.dot(eigenvectors,D)
    X_dagger = X.conjugate().T
    return X, X_dagger

# Parameters and initializations
n = 2
m = 4
Z = 2.0
re = -2.904
al = 137.03599
zeta = np.array([1.45363, 2.91093])

P = np.zeros((m, m))
S = np.zeros((m, m))
H = np.zeros((m, m))
F = np.zeros((m, m))
T = np.zeros((n, n, n, n))
TLS = np.zeros((n, n, n, n))
TSS = np.zeros((n, n, n, n))

# Compute overlap matrix S
for i in range(n):
    for j in range(n):
        S[i, j] = (8.0 * (zeta[i] * zeta[j]) ** 1.5) / (zeta[i] + zeta[j]) ** 3.0

for i in range(n, m):
    for j in range(n, m):
        S[i, j] = (8.0 * (zeta[i-n] * zeta[j-n]) ** 1.5) / (zeta[i-n] + zeta[j-n]) ** 3.0

# Compute Hamiltonian matrix H
for i in range(n):
    for j in range(n):
        H[i, j] = (-4 * Z * (zeta[j] * zeta[i]) ** 1.5) / (zeta[i] + zeta[j]) ** 2.0

for i in range(n, m):
    for j in range(n):
        H[i, j] = (8* al * (zeta[i-n] ** 2.5) * (zeta[j] ** 1.5)) / ((zeta[i-n] + zeta[j]) ** 3.0)

for i in range(n):
    for j in range(n, m):
        H[i, j] = (8*al* zeta[j-n] ** 2.5 * zeta[i] ** 1.5) / (zeta[i] + zeta[j-n]) ** 3.0

for i in range(n,m):
    for j in range(n, m):
        H[i, j] = ((-16*al**2* (zeta[i-n] * zeta[j-n]) ** 1.5) / (zeta[i-n] + zeta[j-n]) ** 3.0 )- ((4*Z * (zeta[i - n] * zeta[j - n]) ** 1.5) / (((zeta[i - n] + zeta[j - n])) ** 2.0))


# Compute two-electron integral tensor SSSS
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                su = zeta[i] + zeta[j] + zeta[k] + zeta[l]
                mul = zeta[i] * zeta[j] * zeta[k] * zeta[l]
                TSS[i, j, k, l] =  (mul ** 1.5)* ((2/ ((zeta[i] + zeta[k]) ** 4 * (zeta[j] + zeta[l]))) -
                                                    (2 / ((zeta[i] + zeta[k]) ** 2 * su ** 3)) -
                                                    (2 / ((zeta[i] + zeta[k]) ** 3 * su ** 2)) -
                                                    (2 / ((zeta[i] + zeta[k]) ** 4 * su ))-
                                                    (2 / ((zeta[i] + zeta[k]) ** 5 )))

# Compute two-electron integral tensor LLSS
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                su = zeta[i] + zeta[j] + zeta[k] + zeta[l]
                mul = zeta[i] * zeta[j] * zeta[k] * zeta[l]
                TLS[i, j, k, l] =  (8*mul ** 1.5)* ((2/ ((zeta[i] + zeta[k]) ** 4 * (zeta[j] + zeta[l]))) -
                                                    (2 / ((zeta[i] + zeta[k]) ** 2 * su ** 3)) -
                                                    (2 / ((zeta[i] + zeta[k]) ** 3 * su ** 2)) -
                                                    (2 / ((zeta[i] + zeta[k]) ** 4 * su )))

# Compute two-electron integral tensor T LLLL
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                su = zeta[i] + zeta[j] + zeta[k] + zeta[l]
                mul = zeta[i] * zeta[j] * zeta[k] * zeta[l]
                T[i, j, k, l] = 16 * mul ** 1.5 * ((2 / ((zeta[i] + zeta[k]) ** 3 * (zeta[j] + zeta[l]) ** 2)) -
                                                    (2 / ((zeta[i] + zeta[k]) ** 2 * su ** 3)) -
                                                    (2 / ((zeta[i] + zeta[k]) ** 3 * su ** 2)))

X, X_dagger = find_X_and_X_dagger(S, m) #Transformation Matrix


C = np.zeros((m, m))  # Initialize coefficient matrix
# Hartree-Fock Self-Consistent Field iteration
E_prev = 0
crit = 1e-10
Delta = 1.0  # Initial difference to enter the loop
iteration = 0
E=1.0

while Delta > crit and iteration < 100:
    iteration += 1
    G = np.zeros((m, m))
    GLS = np.zeros((m, m))
    GSS = np.zeros((m, m))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    G[i, j] += P[k, l] * (T[i, j, k, l] - T[i, l, k, j]) + P[k+n,l+n]*TLS[i,j,k,l]

    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    GLS[i, j] += P[k+n, l] *(TLS[i,l,k,j])

    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    GSS[i, j] +=   P[k+n, l+n] * (T[i, j, k, l] - T[i, l, k, j])+ P[k, l] * (TLS[i,j,k,l])

    #Fock Matrix
    for i in range(n):
      for j in range(n):
        F[i,j]= H[i,j]+ G[i,j]

    for i in range(n):
      for j in range(n,m):
        F[i,j]= H[i,j]- GLS[i,j-n]

    for i in range(n,m):
      for j in range(n):
        F[i,j]= H[i,j]- GLS[j,i-n]


    for i in range(n,m):
      for j in range(n,m):
        F[i,j]= H[i,j]+ GSS[i-n,j-n]

    oldE= E
    E=0.0
    for i in range(m):
        for j in range(m):
          E+= 0.5*(P[i,j]*(H[i,j]+F[i,j])) #Energy

    er = (abs(E - re) / abs(re)) * 100

    #New eigenvectors and density probability
    FP = np.dot(X_dagger,np.dot(F, X))

    CP = autovector(FP, m)
    C = np.dot(X,CP)

    oldP = P.copy()
    P = np.zeros_like(P)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                P[i, j] += C[k,i]*C[j,k]

    for i in range(n,m):
        for j in range(n):
            for k in range(n):
                 P[i, j] += C[k,i]*C[j,k]


    for i in range(n):
        for j in range(n,m):
            for k in range(n,m):
                 P[i, j] += C[k,i]*C[j,k]


    for i in range(n,m):
        for j in range(n,m):
            for k in range(n,m):
                 P[i, j] += C[k,i]*C[j,k]

    Delta = np.abs(oldE - E)

    print(f"Iteration: {iteration}, Total Energy: {E:.6f}, Real Energy: {re:.6f}, Error: {er:.6f}")


    if Delta < crit:
        break

# Print the final matrices
print("Matriz de Fock")
for i in range(4):
    print(" ".join(f"{H[i, j]:10.3f}" for j in range(4)))

print("C primado")
for i in range(4):
    print(" ".join(f"{CP[i, j]:10.3f}" for j in range(4)))

print("C")
for i in range(4):
    print(" ".join(f"{C[i, j]:10.3f}" for j in range(4)))

print("P")
for i in range(4):
    print(" ".join(f"{P[i, j]:10.3f}" for j in range(4)))





Iteration: 1, Total Energy: 0.000000, Real Energy: -2.904000, Error: 100.000000
Iteration: 2, Total Energy: -17.288737, Real Energy: -2.904000, Error: 495.342170
Iteration: 3, Total Energy: -17.289085, Real Energy: -2.904000, Error: 495.354170
Iteration: 4, Total Energy: -17.289085, Real Energy: -2.904000, Error: 495.354170
Iteration: 5, Total Energy: -17.289085, Real Energy: -2.904000, Error: 495.354170
Matriz de Fock
    -2.907     -3.655    199.200    334.090
    -3.655     -5.822    166.834    398.902
   199.200    166.834 -37560.632 -31459.136
   334.090    398.902 -31459.136 -37563.547
C primado
     0.007     -0.004      0.997      0.081
    -0.793      0.609      0.007      0.010
    -0.012     -0.001      0.081     -0.997
     0.609      0.793     -0.000     -0.008
C
     0.006     -0.008      1.791     -0.378
    -0.018      0.007     -1.706     -0.661
    -1.073      1.482      0.013      0.013
     1.709     -0.655     -0.013     -0.021
P
     0.000     -0.000      0.008   